In [2]:
!git clone https://github.com/MeesLursen/KernelLTL.git

fatal: destination path 'KernelLTL' already exists and is not an empty directory.


In [11]:
!cd /kaggle/working/KernelLTL && git pull && cd ..

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 267 bytes | 267.00 KiB/s, done.
From https://github.com/MeesLursen/KernelLTL
   06f9ebf..f8bb618  main       -> origin/main
Updating 06f9ebf..f8bb618
Fast-forward
 train.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [4]:
import sys
sys.path.insert(0, "/kaggle/working/KernelLTL") 

In [5]:
!wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
!./bin/micromamba create -y -n kernelltl -c conda-forge python=3.13.7 pip
!./bin/micromamba run -n kernelltl pip install -r /kaggle/working/KernelLTL/requirements.txt

bin/micromamba
[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━━━╸━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
conda-forge/noarch   ━━━━━━━━━━━━━━━━━━━━━━━  14.1kB /  22.8MB @ 193.0kB/s  0.0s[+] 0.2s
conda-forge/linux-64 ╸━━━━━━━━━━━━━━━━━━━━━━   4.1MB /  47.8MB @  23.3MB/s  0.1s
conda-forge/noarch   ━━━╸━━━━━━━━━━━━━━━━━━━   4.3MB /  22.8MB @  24.3MB/s  0.1s[+] 0.3s
conda-forge/linux-64 ━━━╸━━━━━━━━━━━━━━━━━━━   9.0MB /  47.8MB @  32.0MB/s  0.2s
conda-forge/noarch   ━━━━━━━━╸━━━━━━━━━━━━━━   9.2MB /  22.8MB @  32.4MB/s  0.2s[+] 0.4s
conda-forge/linux-64 ━━━━━╸━━━━━━━━━━━━━━━━━  13.8MB /  47.8MB @  35.8MB/s  0.3s
conda-forge/noarch   ━━━━━━━━━━━━━╸━━━━━━━━━  13.9MB /  22.8MB @  36.0MB/s  0.3s[+] 0.5s
conda-forge/linux-64 ━━━━━━━╸━━━━━━━━━━━━━━━  18.7MB /  47.8MB @  38.2MB/s  0.4s
conda-forge/noarch   ━━━━━━━━━━━━━━━━━╸━━━━━  18.9MB /  22.8MB @  38.3MB/s  0.4s[+] 0.6s
conda-forge/linux-64 ━━━━━━━━━╸━━━━━━━━━━━━━  21.2MB /  47.8MB @  39.0MB/s  0.5s
conda-forge/noarch   ━━━━━━━━━━━━━━━

In [12]:
!cd /kaggle/working/KernelLTL && \
 PYTHONPATH=$PWD /kaggle/working/bin/micromamba run -n kernelltl \
 torchrun --nproc_per_node=2 -m train

W1012 18:50:30.954000 424 site-packages/torch/distributed/run.py:774] 
W1012 18:50:30.954000 424 site-packages/torch/distributed/run.py:774] *****************************************
W1012 18:50:30.954000 424 site-packages/torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1012 18:50:30.954000 424 site-packages/torch/distributed/run.py:774] *****************************************
/root/.local/share/mamba/envs/kernelltl/lib/python3.13/site-packages/torch/distributed/algorithms/ddp_comm_hooks/__init__.py:54: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in enum.member() if you want to preserve the old behavior
  ALLREDUCE = partial(_ddp_comm_hook_wrapper, comm_hook=default.allreduce_hook)
/root/.local/share/mamba/envs/kernelltl/lib/python3.13/site-pa

In [ ]:
import math
import os
import torch

# Hyperparameters
num_epochs = 10

learning_rate = 5e-5

T       = 20
AP      = 5
seed    = 1

eps     = 0.01
delta   = 1 - 0.99
N       = math.ceil((2 / eps**2) * math.log(2 / delta))

m       = 1024
    
# Create output directory
output_dir = "ltl_model_outputs"
os.makedirs(output_dir, exist_ok=True)
local_rank = int(os.environ.get("LOCAL_RANK", -1))
if local_rank != -1 and torch.cuda.is_available():
    torch.cuda.set_device(local_rank)
# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
# torch.use_deterministic_algorithms(True)

In [ ]:
from transformers import TrainingArguments, Trainer
from tokenizer_class import LTLTokenizer
from kernel_class import LTLKernel
from dataset_class import LTLDataset

# Initialize tokenizer (adjust n_ap based on your needs)
tokenizer = LTLTokenizer(n_ap=AP)

# Initialize kernel for semantic embeddings
kernel = LTLKernel(T, AP, seed)  # adjust T and AP as needed
kernel.sample_traces_kernel(N)  # adjust N based on your needs
kernel.sample_anchor_formulas_kernel(m)  # m should match model's n_embd
kernel.build_F()

# Create datasets
train_dataset = LTLDataset()
train_dataset.construct_dataset_from_kernel(
    kernel=kernel,
    k=78000,  # adjust dataset size as needed
    p_leaf=0.45,
    max_depth=1000,
    batch_size=10240
)

eval_dataset = LTLDataset()
eval_dataset.construct_dataset_from_kernel(
    kernel=kernel,
    k=1000,  # smaller validation set
    p_leaf=0.45,
    max_depth=1000,
    batch_size=10240
)

In [ ]:
from model_class import LTLModel
from config_class import LTLConfig
from training_utils import SemanticEvaluationCallback

# Create model configuration and model
config = LTLConfig(
    n_embd=m  # must match kernel's anchor set size (m)
)

model = LTLModel(config, semantic_emb_dim=m)  # semantic_emb_dim must match kernel's anchor set size

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f"{output_dir}/logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    ddp_find_unused_parameters=False
)

# Initialize callback
semantic_callback = SemanticEvaluationCallback(
    kernel=kernel,
    tokenizer=tokenizer
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=lambda batch : tokenizer.collate_batch(batch, model.config.n_positions),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    callbacks=[semantic_callback]
)

In [ ]:
# Train
trainer.train()

# Save final model
trainer.save_model(os.path.join(output_dir, "final_model"))
tokenizer.save_vocab(os.path.join(output_dir, "vocab.json"))